# Getting MBTI tagged posts from Reddit

Created by: Sara Jakša

First we are importing the necessary libaries. There are just things to access Reddit API this time.

In [1]:
import praw
from praw.models import MoreComments

This is going to be the file we are going to save our results in.

In [2]:
filename = "reddit.csv"

This are the credentials needed to access Reddit data. You can get it here: [https://www.reddit.com/prefs/apps](https://www.reddit.com/prefs/apps). 

In [12]:
client_id=""
client_secret=""
user_agent=""

This are types names that we are going to be using for comparison and accessing subreddits. 

In [5]:
typenames = ["ENFJ",
             "ENFP",
             "ENTJ",
             "ENTP",
             "ESFJ",
             "ESFP",
             "ESTJ",
             "ESTP",
             "INFJ",
             "INFP",
             "INTJ",
             "INTP",
             "ISFJ",
             "ISFP",
             "ISTJ",
             "ISTP"]

This is a client, that will enable us to get Reddit data.

In [6]:
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)

This is a helper function that will phrase the data needed.

In [7]:
def getinfofromdata(post):
    postid = post.id
    if post.author:
        author = post.author.name
    else:
        return None
    try:
        text = post.selftext
    except AttributeError:
        text = post.body
    text = text.replace("\n", " ").replace("\t", " ")
    if post.author_flair_text:
        typeofpost = post.author_flair_text
    else:
        return None
    if not text.strip():
        return None
    return postid, author, typeofpost, text

This function allows us to access subreddit and get their submissions and comments on these submissions.

In [8]:
def functionformbti(typename):
    alltexts = list()

    for submission in reddit.subreddit(typename).new(limit=None):
        alltexts.append(getinfofromdata(submission))
        for comment in submission.comments:
            if isinstance(comment, MoreComments):
                continue
            alltexts.append(getinfofromdata(submission))
    return list(set(alltexts))

Now we are going to access all of the subreddits that we are interested in.

In [10]:
alltexts = []
for typename in typenames + ["MBTI"]:
    texts = functionformbti(typename.lower())
    texts = list(set(texts))
    alltexts.append([text for text in texts if text and text[2].upper().strip() in typenames])

And now we will write the results in the file.

In [11]:
with open(filename, "a", -1, "utf-8") as write:
    for typename in alltexts:
        for text in typename:
            write.write("\t".join(text) + "\n")